This is a simple example on how to optimize Bell inequalities on the set of quantum correlations using [ncpol2sdpa](https://ncpol2sdpa.readthedocs.io/)

In [1]:
from ncpol2sdpa import flatten, SdpRelaxation, generate_operators, \
                       projective_measurement_constraints

In [2]:
def generate_noncommuting_measurements(party, label):
    """Generates the list of symbolic variables representing the measurements
    for a given party. The variables are treated as noncommuting.

    :param party: configuration indicating the configuration of number m
                  of measurements and outcomes d for each measurement. It is a
                  list with m integers, each of them representing the number of
                  outcomes of the corresponding  measurement.
    :type party: list of int
    :param label: label to represent the given party
    :type label: str

    :returns: list of sympy.core.symbol.Symbol
    """
    measurements = []
    for i, p in enumerate(party):
        measurements.append(generate_operators(label + '_%s' % i, p - 1,
                                               hermitian=True))
    return measurements

First, we define how many inputs and outputs each party will have, and define a set of projectors for each of them

In [3]:
oA = [2, 2]    # Number of outputs of each of Alice's measurements
oB = [2, 2]    # Number of outputs of each of Bob's measurements

In [8]:
# Alice's measurements
measA = generate_noncommuting_measurements(oA, 'A')
        
# Bob's measurements
measB = generate_noncommuting_measurements(oB, 'B')

substitutions = projective_measurement_constraints([measA, measB])

Instead of projectors, it is (in this case) more convenient to have the operators represent quantum observables. We do this by changing the substitutions for the square of the operators.

In [8]:
for measurement in flatten(measA+measB):
    substitutions[measurement**2] = 1

Now we can define the Bell operator one wants to optimize

In [9]:
CHSH = (measA[0][0]*(measB[0][0] + measB[1][0])
        + measA[1][0]*(measB[0][0] - measB[1][0]))

And finally, generate the SDP relaxation of the problem and solve it

In [13]:
sdp = SdpRelaxation(flatten([measA, measB]), verbose=2)
sdp.get_relaxation(level=1,                        # Initial NPA level of the moment matrix
                   objective=-CHSH,                # Objective function
                   substitutions=substitutions     # Idempotency and commutation rules
                   )
sdp.solve("mosek")
print(sdp.primal, sdp.dual)

The problem has 4 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 14
Generating moment matrix...
Reduced number of SDP variables: 10 10 (done: 107.14%, ETA 00:00:-0.0)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10              
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - num